### 图像金字塔   
在图像处理中常常会调整图像大小，最常见的就是放大(zoom in)和缩小(zoom out)，尽管几何变换也可以实现图像放大和缩小（如双线性差值、双类方差值等）。  
  
但是图像金字塔的上采样和降采样可以得到下一层和上一层不同分辨率的图像，以在不同的尺度空间去做特征处理。一个图像金字塔是由一系列的图像组成，最底下一张是图像尺寸最大，最上方的图像尺寸最小。  
  
图像金字塔可用在图像融合、重建、特征提取等方面


图像的采样点越多，像素越多，分辨率越高。

In [4]:
import cv2
def cv_show(name,img):
    cv2.imshow(name,img)
    while cv2.waitKey(100) != 27:# loop if not get ESC
        if cv2.getWindowProperty(name,cv2.WND_PROP_VISIBLE) <= 0:
            break
    cv2.destroyWindow(name)

In [5]:
img = cv2.imread('dog_3.jpeg')
cv_show('orig',img)
print(img.shape)

(406, 650, 3)


### 高斯金字塔：向上采样方法（放大）  
1.将图像在每个方向扩大为原来的两倍，新增的行和列以0填充。  
2.使用先前同样的内核(乘以4)与放大后的图像卷积，获得近似值。   

In [11]:
up = cv2.pyrUp(img)
cv_show('orig',img)
print('up',up.shape)

up (812, 1300, 3)


### 高斯金字塔(Gaussianpyramid) – 用来对图像进行降采样  
高斯金子塔是从底向上，逐层降采样得到（不能跳层）。降采样之后图像大小是原图像MxN的M/2 x N/2 ,就是对原图像删除偶数行与列，即得到降采样之后上一层的图片。  
  
高斯金子塔的生成过程分为两步：  

1）对当前层进行高斯模糊  

2）删除当前层的偶数行与列  

即可得到上一层的图像，这样上一层跟下一层相比，都只有它的1/4大小。


In [12]:
down = cv2.pyrDown(img)
cv_show('orig',img)
print('down',down.shape)

down (203, 325, 3)


### 值得注意的是，先对图像进行上采样，再进行下采样，原图信息仍会丢失，即是说对图像进行下采样一定会导致信息的丢失！

In [14]:
up = cv2.pyrUp(img)
up_down = cv2.pyrDown(up)
import numpy as np 
res = np.hstack((img,up_down))
cv_show('res',res)

### 拉普拉斯金字塔(Laplacianpyramid)  
用来从金字塔低层图像重建上层未采样图像，在数字图像处理中也即是预测残差，可以对图像进行最大程度的还原，配合高斯金字塔一起使用。  
两者的简要区别：高斯金字塔用来向下降采样图像，而拉普拉斯金字塔则用来从金字塔底层图像中向上采样重建一个图像。  
求法：先对原图像进行下采样，再进行上采样，使用原图像相减，即可得到丢失的信息。

In [8]:
down = cv2.pyrDown(img)
down_up = cv2.pyrUp(down)
l = img - down_up
cv_show('laplacian',l)

也就是说，拉普拉斯金字塔是通过源图像减去先缩小后再放大的图像的一系列图像构成的。
